In [6]:
import lib.PrintImg
import lib.DataInput

In [8]:
file1 = "0043db5e8c819bffc15261b1f1ac5e42.aps"
file2 = "00360f79fd6e02781457eda48f85da90.aps"
lib.PrintImg.save_images("../sample_data/"+file2,"../sample_image/"+file2)

In [9]:
data = lib.DataInput.read_data("../sample_data/"+file2)

In [11]:
data.shape

(512, 660, 16)